# python环境下基于socket的raspberry与PC的视频传输

## 准备

* 硬件要求
 * 接入局域网的Raspberry B+
 * 接入局域网的PC

* 软件要求
 * Raspberry B+: 装有opencv库的python3
 * PC：装有opencv库的python3，在windows系统测试过，linux系统没有

## 软件实现

### 知识介绍

* socket简介：socket是Socket是进程通讯的一种方式，即调用这个网络库的一些API函数实现分布在不同主机的相关进程之间的数据交换
* 相关概念：IP地址，端口号，TCP/IP通信协议
 * 端口号：每一个本地的进程在通讯时均会占用一个端口号，不同的进程端口号不同，不同的端口号区分不同的进程。
 * TCP协议：（Transmission Control Protocol，传输控制协议）是面向连接的通信协议。一般的通信流程需要经历以下几个步骤：新建套接字，连接，监听，发送，接受。TCP相比UDP的流程更加复杂，但通讯质量更加稳定，一般应用与对可靠性要求较高的场合。
 * UDP协议：（User Data Protocol，用户数据报协议）是面向非连接的协议，它的特点是不与对方建立连接，而是直接将数据包发送过去。一般应用于对可靠性要求不高的场合。

### 算法难点

* 编码：将numpy的数组类型转换成发送函数的发送数据的参数类型
* 解码：将接受函数接受到的数据类型转换成opencv库函数可以使用的numpy的数组类型

### 客户端python源码 

导入必要的库

In [ ]:
import socket
import cv2
import numpy

建立套接字，并绑定相应的端口和IP地址，其中IP地址为服务器端，即PC端的IP地址

In [ ]:
address = ('192.168.1.109', 8002)
sock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
sock.connect(address)

开启摄像头，由于树莓派的性能相对来说较差，考虑到帧率问题，将每帧的像素大小设为320×240，并读取第一帧图像

In [ ]:
capture = cv2.VideoCapture(0)
capture.set(3, 320)
capture.set(4, 240)
ret, frame = capture.read()

设置编码参数

In [ ]:
encode_param=[int(cv2.IMWRITE_JPEG_QUALITY),90]

进入循环，按下ESC键退出

In [ ]:
while ret:    

显示待发送的图像

In [ ]:
    cv2.imshow('CLIENT',frame)
    if cv2.waitKey(10) == 27:
        break    

编码

In [ ]:
    result, imgencode = cv2.imencode('.jpg', frame, encode_param)

转化成字符串格式

In [ ]:
    data = numpy.array(imgencode)
    stringData = data.tostring()    

发送数据长度

In [ ]:
    sock.send( str(len(stringData)).ljust(16).encode('utf-8'))

发送完整数据

In [ ]:
    sock.send( stringData )

读取图像

In [ ]:
    ret, frame = capture.read()

当退出循环后，释放资源，分别释放sock，capture，以及显示窗口

In [ ]:
sock.close()
capture.release()
cv2.destroyAllWindows()

### 服务端python源码

导入必要的库

In [ ]:
import socket
import cv2
import numpy

自定义数据接受函数

In [ ]:
def recvall(sock, count):
    buf = b''
    while count:
        newbuf = sock.recv(count)
        if not newbuf: return None
        buf += newbuf
        count -= len(newbuf)
    return buf

建立套接字，并绑定IP地址和端口号

In [ ]:
address = ('192.168.1.109', 8002)
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(address)

开始监听

In [ ]:
sock.listen(True)

连接后获取客户端的IP地址等信息

In [ ]:
conn, addr = sock.accept()

进入循环

In [ ]:
while True:

获取数据长度

In [ ]:
    length = recvall(conn,16)

获取完整数据

In [ ]:
    stringData = recvall(conn, int(length))

将字符串转换成编码变量

In [ ]:
    data = numpy.fromstring(stringData, dtype='uint8')

将编码变量转换成图像变量

In [ ]:
    decimg=cv2.imdecode(data,1)

显示图像

In [ ]:
    cv2.imshow('SERVER',decimg)
    if cv2.waitKey(10) == 27:
        break

释放相应资源

In [ ]:
s.close()
cv2.destroyAllWindows()